<a href="https://colab.research.google.com/github/solus9/OSML/blob/master/fellowship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FellowShip.AI

In [4]:
import os
import numpy as np
import imageio
import pickle

import numpy as np
from sklearn.utils import shuffle

import numpy as np
import pickle
import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.models import Model

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K
from sklearn.utils import shuffle

#### data_loader.py

In [6]:
class DataLoader:
	def __init__(self, data_path, batch_size=10, image_width=105, image_height=105):
		self.data_path = data_path
		self.train_paths_dictionary = {}
		self.evaluation_paths_dictionary = {}
		self.image_width = image_width
		self.image_height = image_height
		self.batch_size = batch_size
		self.training_data = None
		self.evaluation_data = None

		self.load_data_paths()

	def load_data_paths(self, ):
		train_path = os.path.join(self.data_path, 'images_background')
		evaluation_path = os.path.join(self.data_path, 'images_evaluation')

		for alphabet in os.listdir(train_path):
			print('[Training Data Paths] loading paths : {}'.format(alphabet))
			alphabet_path = os.path.join(train_path, alphabet)
			alphabet_dictionary = {}
			for character in os.listdir(alphabet_path):
				character_paths = os.path.join(alphabet_path, character)
				alphabet_dictionary[character] = os.listdir(character_paths)
			self.train_paths_dictionary[alphabet] = alphabet_dictionary
		
		for alphabet in os.listdir(evaluation_path):
			print('[Evaluation Data Paths] loading paths : {}'.format(alphabet))
			alphabet_path = os.path.join(evaluation_path, alphabet)
			alphabet_dictionary = {}
			for character in os.listdir(alphabet_path):
				character_paths = os.path.join(alphabet_path, character)
				alphabet_dictionary[character] = os.listdir(character_paths)
			self.evaluation_paths_dictionary[alphabet] = alphabet_dictionary

	def load_images(self, train=True):
		alphabets_to_categories = {}
		X = []
		y = []
		# categories_to_alphabet_character = {}
		current_class = 0
		paths_dictionary = self.train_paths_dictionary if train is True else self.evaluation_paths_dictionary
		data_path = os.path.join(self.data_path, 'images_background') if train is True \
			else os.path.join(self.data_path, 'images_evaluation')
		for alphabet in paths_dictionary:
			print('[{} Data] Loading Images : {}'.format('Training' if train is True else 'Test', alphabet))
			alphabets_to_categories[alphabet] = [current_class, None]
			for character in paths_dictionary[alphabet]:
				# categories_to_alphabet_character[current_class] = [alphabet, character]
				category_images = []
				for file in os.listdir(os.path.join(data_path, alphabet, character)):
					image = imageio.imread(os.path.join(data_path, alphabet, character, file))
					category_images.append(image)
					y.append(current_class)
				try:
					X.append(np.stack(category_images))
				except ValueError as e:
					print(e)
					print("error : catergory images : {}".format(category_images))
				current_class += 1
			alphabets_to_categories[alphabet][1] = current_class - 1
		y = np.vstack(y)
		X = np.stack(X)
		return X, y, alphabets_to_categories

	def load_data(self, ):
		self.load_data_paths()
		self.training_data = self.load_images()
		self.evaluation_data = self.load_images(train=False)

	def store_data(self, path=None):
		if path is None: path = self.data_path
		with open(os.path.join(path, 'training_data.pkl'), 'wb') as writeFile:
			pickle.dump(self.training_data, writeFile)
		print('Training Data written : {}'.format(os.path.join(path, 'training_data.pkl')))
		with open(os.path.join(path, 'evaluation_data.pkl'), 'wb') as writeFile:
			pickle.dump(self.evaluation_data, writeFile)
		print('Evaluation Data written : {}'.format(os.path.join(path, 'evaluation_data.pkl')))

#### batch_generatory.py

In [7]:
class BatchManager:
	def __init__(self, loaded_data):
		self.width = 105
		self.height = 105
		(self.X, self.y, self.categories) = loaded_data
		
	def get_batch(self, batch_size=1000):
		classes, items_per_class, width, height = self.X.shape
		chosen_categories = np.random.choice(classes, size=(batch_size, ), replace=False)

		pairs = [np.zeros((batch_size, width, height, 1)) for _ in range(2)]

		targets = np.zeros((batch_size, ))
		targets[batch_size//2:] = 1

		count = 0
		for category in chosen_categories:
			chosen_item = np.random.randint(0, items_per_class)
			pairs[0][count, :, :, :] = self.X[category, chosen_item].reshape(width, height, 1)
			if count < batch_size // 2:
				category = (category + np.random.randint(1, classes)) % classes
				chosen_item = np.random.randint(0, items_per_class)
			else:
				chosen_item = (chosen_item + np.ranint(1, items_per_class)) % items_per_class
			pairs[1][count, :, :, :] = self.X[category, chosen_item].reshape(width, height, 1)
		return pairs, targets

	def generator(self, batch_size):
		while True:
			pairs, targets = self.get_batch(batch_size)
			yield (pairs, targets)
   
  def get_test_batch(self, batch_size):
		classes, items_per_class, width, height = self.X.shape
		indices = np.random.randin(0, items_per_class, size=(batch_size, ))
		chosen_categories = np.random.choice(range(classes), size=(batch_size, ), replace=False)
		true_category = chosen_categories[0]
		item1, item2 = np.random.choice(items_per_class, size=(2, ), replace=False)
		# create N copies for the true category item for evaluation in 'batch_size' way
		test_image = np.asarray([self.X[true_category, item1, : , :]] * batch_size).reshape( \
			batch_size, width, height, 1)
		suuport_images = X[chosen_categories, indices, :, :]
		support_images[0, :, :] = X[true_category, item2]
		support_images = support_images.reshape(batch_size, self.width, self.height, 1)
		targets = np.zeros(size(batch_size, ))
		targets[0] = 1
		targets, test_image, support_images = shuffle(targets, test_image, support_images)
		pairs = [test_image, support_images]
		return pairs, targets

IndentationError: ignored

#### siamese.py

In [8]:
def initialize_weights(shape):
	return np.random.normal(loc=0.0, scale=1e-2, size=shape)

def initialize_bias(shape):
	return np.random.normal(loc=0.5, scale=1e-2, size=shape)

def get_siamese_model(input_shape):
	left_input = Input(input_shape)
	right_input = Input(input_shape)

	model = Sequential()
	model.add(Conv2D(64, (10, 10), activation='relu', input_shape=input_shape, \
		kernel_initializer=initialize_weights, kernel_regularizer=12(2e-4)))
	model.add(MaxPooling2D())
	model.add(Conv2D(128, (7, 7), activation='relu', \
		kernel_initializer=initialize_weights, bias_initializer=initialize_bias, \
		kernel_regularizer=12(2e-4)))
	model.add(MaxPooling2D())
	model.add(Conv2D(128, (4, 4), activation='relu', \
		kernel_initializer=initialize_weights, bias_initializer=initialize_bias, \
		kernel_regularizer=12(2e-4)))
	model.add(MaxPooling2D())
	model.add(Conv2D(256, (4, 4), activation='relu', \
		kernel_initializer=initialize_weights, bias_initializer=initialize_bias, \
		kernel_regularizer=12(2e-4)))
	model.add(Flatten())
	model.add(Dense(4096, activation='sigmoid', kernel_regularizer=12(1e-3), \
		kernel_initializer=initialize_weights, bias_initializer=initialize_bias))
	left_encoded = model(left_input)
	right_encoded = model(right_input)

	L1_layer = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
	L1_distance = L1_layer([left_encoded, right_encoded])

	prediction = Dense(1, activation='sigmoid', bias_initializer=initialize_bias)(L1_distance)
	siamese_network = Model(inputs=[left_input, right_input], output=prediction)
	return siamese_network

def test_oneshot_once(model, validataion_data, N, verbose=True):
	pairs, targets = validataion_data
	if targets.shape < N:
		raise ValueError('supported images are : {} which is less than \
			Classification Way: {}'.format(pairs[1].shape, N))
	if verbose:
		print('Evaluating model on validation set with random {} way one-shot learning tasks...'.format(N))
	probabilities = model.predict(pairs)
	if np.argmax(probabilities) == np.argmax(targets):
		return True
	else: return False

#### training.py